In [1]:
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [2]:
from pymongo import MongoClient
client = MongoClient("mongodb://10.120.37.108:27017")
db = client["fb_cleaned"]
collection = db["message_cleaned"]
messages_score = list(collection.find({},{"_id":0,"politician_id":0}))

In [3]:
messages_list = [m for m in messages_score if len(m["message"]) >=20]

In [4]:
len(messages_list)

18006

In [5]:
import pandas as pd
df = pd.DataFrame(messages_list)
text = df["message"].values
y = df["score"].values

In [6]:
text

array([ '稅收 讓利 企業 誘使 企業 員工 直接 加薪 雙面 有利 很好 將心 比心 企業 短入 荷包 分享 員工 政府 讓利 企業 很好 方法 其實 簡單 薪資 費用 加乘 倍率 承認 薪資 費用 薪資 費用 達到 收入 申報 減乘 計算 薪資 費用 勞健 保險費 一律 薪資 費用 投保 導正 海外 就業 人口 依未 境內 天數 來加 加乘 費率 計收 保險費 導正 不公平 健保',
       '對馬 弊案 有點 進展 值得 稱讚 根本 進展 樂聲 弊案 銀行 洗錢 百多 慶富 關心 一粒 一休 企業 起薪 企業 老闆 不肯 賴清德 縮回去 下文',
       '賴清德 加油 希望 完善 便捷 公車 運輸 系統 搭車 價格 騎乘 機車 太多 吸引 民眾 騎乘 機車 搭乘 公車 轉換 捷運 輕軌 公共 運輸 系統 減少 發生 機車 死亡 車禍 機會 觀光 遊憩 資源 吸引 更多 海外 觀光客 旅行 消費 購物 帶動 當地 經濟 開創 更多 賺錢 機會',
       ...,
       '其實 覺得 希望 民眾 自發 減塑 政府 方面 有所 塑膠袋 塑膠 吸管 衛生 一次性 用品 製造 廠商 課徵 環保 越重 越好 香煙 課徵 健康捐 廠商 吸收 不了 自然 提高 售價 民眾 伸手 一個 塑膠袋 付出 相信 應該 有效 減低 塑膠 使用量',
       '出門 塑膠袋 看到 身邊 塑膠袋 收起 洗乾淨 分給 需要 攤販 重複 大樓 清潔員 垃圾袋 使用 覺得 袋子 沒壞 丟了 可惜 希望 身邊 有所 做到 塑膠袋',
       '感謝 蘇麗瓊 執行長 柯文哲 你還 抹黑 執行長'], dtype=object)

In [7]:
from sklearn.model_selection import train_test_split
train_text, test_text, y_train, y_test = train_test_split(text, y, test_size=0.2, random_state=42)

In [67]:
token = Tokenizer(num_words=1000)
token.fit_on_texts(train_text)

In [68]:
token.document_count

21625

In [71]:
len(token.word_index)

41703

In [72]:
x_train_seq = token.texts_to_sequences(train_text)
x_test_seq = token.texts_to_sequences(test_text)

In [73]:
train_text[50]

'好像 一個 國民黨 上台 執行 真的 失望'

In [74]:
print(x_train_seq[50])

[284, 14, 60, 179, 6, 174]


In [19]:
x_train = sequence.pad_sequences(x_train_seq, maxlen=20)
x_test = sequence.pad_sequences(x_test_seq, maxlen=20)

In [20]:
print("before:",len(x_train_seq[0]))

before: 3


In [21]:
print("after:",len(x_train[0]))

after: 20


# MLP

In [25]:
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
token = Tokenizer(num_words=10000)
token.fit_on_texts(train_text)
x_train_seq = token.texts_to_sequences(train_text)
x_test_seq = token.texts_to_sequences(test_text)
x_train = sequence.pad_sequences(x_train_seq, maxlen = 100)
x_test = sequence.pad_sequences(x_test_seq, maxlen = 100)

In [26]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding

model = Sequential()

model.add(Embedding(output_dim=32,
                    input_dim=10000,
                    input_length=100))
model.add(Dropout(rate = 0.2))

model.add(Flatten())

model.add(Dense(units=256,
                activation="relu"))
model.add(Dropout(rate = 0.35))

model.add(Dense(units=1,
                activation="sigmoid"))
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 32)           320000    
_________________________________________________________________
dropout_3 (Dropout)          (None, 100, 32)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               819456    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 1,139,713
Trainable params: 1,139,713
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.compile(loss="binary_crossentropy",
              optimizer = "adam",
              metrics=["accuracy"])
model.fit(x_train, 
          y_train, 
          batch_size=1000,
          epochs=10, 
          verbose=2,
          validation_split=0.2)

Train on 23637 samples, validate on 5910 samples
Epoch 1/10
 - 3s - loss: 0.6570 - acc: 0.5983 - val_loss: 0.6084 - val_acc: 0.6641
Epoch 2/10
 - 3s - loss: 0.4770 - acc: 0.7989 - val_loss: 0.3648 - val_acc: 0.8459
Epoch 3/10
 - 3s - loss: 0.2804 - acc: 0.8876 - val_loss: 0.2937 - val_acc: 0.8716
Epoch 4/10
 - 3s - loss: 0.2127 - acc: 0.9139 - val_loss: 0.3042 - val_acc: 0.8679
Epoch 5/10
 - 3s - loss: 0.1813 - acc: 0.9290 - val_loss: 0.3097 - val_acc: 0.8687
Epoch 6/10
 - 3s - loss: 0.1596 - acc: 0.9388 - val_loss: 0.3203 - val_acc: 0.8660
Epoch 7/10
 - 3s - loss: 0.1397 - acc: 0.9474 - val_loss: 0.3461 - val_acc: 0.8650
Epoch 8/10
 - 3s - loss: 0.1246 - acc: 0.9520 - val_loss: 0.3596 - val_acc: 0.8607
Epoch 9/10
 - 3s - loss: 0.1149 - acc: 0.9565 - val_loss: 0.3825 - val_acc: 0.8613
Epoch 10/10
 - 3s - loss: 0.1078 - acc: 0.9585 - val_loss: 0.4038 - val_acc: 0.8591


In [10]:
scores = model.evaluate(x_test, y_test, verbose=1)
scores[1]

12663/12663 [==============================] - 0s 22us/step


0.85864329142733542

In [11]:
prediction = model.predict_classes(x_test)
prediction[:10]
prediction_classes=prediction.reshape(-1)
prediction_classes[:10]

array([0, 0, 0, 1, 0, 1, 1, 0, 0, 0])

In [12]:
SentimentDict={1:"P",0:"N"}
def display_test_Sentiment(i):
    print(test_text[i])
    print("label真實值",SentimentDict[y_test[i]])
    print("預測值",SentimentDict[prediction_classes[i]])

In [13]:
display_test_Sentiment(1)

勞工 拜託 加薪 加班 無良 企業 竄改 班表 業界 常態 員工 根本 領不到 加班費 屢次 違法 老闆 刑事 責任 放入 勞基法
label真實值 N
預測值 N


In [14]:
display_test_Sentiment(120)

院长 辛苦
label真實值 N
預測值 N


In [64]:
input_text=["賴清德 江宜樺 位子 腦 權力 就 不理會 人民 蔡英文 用人不當 民進黨 失望透頂 年底 選舉 民進黨 步上 國冥黨 後塵",
            "不知 執政黨 有何 對策 終結 亂象 總而言之 習慣 就好 民眾 提案 禁 五星旗 法務部 不符 憲法 保障 人民 言論 自由 意旨 提案 不予 採納 金門老街 出現 景點 知名 模範街 同時 出現 中華民國 國旗 中國 五星旗 旅台鄉親 我們 厦門 習慣 看到 五星旗 元旦 五星旗 彰化 共匪窩 國安局 鎖定 西門町 六號 出口 台獨 駐點 今天 淪陷 五星旗 插滿 人行道 侵門踏戶 中國 央視 台北 西門町 大登 廣告 陸機 繞台 中共 連喊 常態化 習慣就好 邱太三 下台",
            "整天 想著 企業 鬆綁 降低 稅賦 最後 經濟 果實 看到 勞工 身上 美國 稅改 政策 蠢蠢欲動 勞工低薪 只會 呼籲 廢物",
            """皮鞋 拖鞋 酒駕 撞 死人 法律 想想 自己 立法 酒駕 政權 跨台""",
            """賴院長 已經 不信任 當初 慶富案""",
            """奴隸 勞工 缺德 政府 不得 民心 枉然""",
            """賴清 沒用 行不行 資進黨 橡皮圖章 黑心 立委 財團 老闆 不開心 怎麼辦""",
            """稅改 賴屎 亂修 台灣 老闆 爛 政客 繼續 吃香喝辣 賴屎""",
            """請問 預計 幾點 結束 昨晚 放炮 嬰兒 嚇的 整夜""",
            """讚 好棒 加油""",
           ]

In [65]:
input_seq = token.texts_to_sequences(input_text)

In [67]:
pad_input_seq = sequence.pad_sequences(input_seq, maxlen=20)

In [ ]:
predict = model.predict_classes(pad_input_seq)

In [61]:
predict

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0]])

In [62]:
predict[0][0]

0

In [63]:
SentimentDict[predict[0][0]]

'N'

# RNN

In [8]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN

In [9]:
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
token = Tokenizer(num_words=40000)
token.fit_on_texts(train_text)
x_train_seq = token.texts_to_sequences(train_text)
x_test_seq = token.texts_to_sequences(test_text)
x_train = sequence.pad_sequences(x_train_seq, maxlen = 250)
x_test = sequence.pad_sequences(x_test_seq, maxlen = 250)

In [10]:
model = Sequential()
model.add(Embedding(output_dim=32,
                    input_dim =40000,
                    input_length = 250))
model.add(Dropout(0.25))
model.add(SimpleRNN(units = 16))

model.add(Dense(units= 256, activation= 'relu'))
model.add(Dropout(0.25))
model.add(Dense(units=1, activation='sigmoid'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 250, 32)           1280000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 250, 32)           0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 16)                784       
_________________________________________________________________
dense_1 (Dense)              (None, 256)               4352      
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 1,285,393
Trainable params: 1,285,393
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer = "adam",
              metrics=["accuracy"])
model.fit(x_train, 
          y_train, 
          batch_size=1000,
          epochs=8, 
          verbose=2,
          validation_split=0.2)
scores = model.evaluate(x_test, y_test, verbose=1)
scores[1]

Train on 11523 samples, validate on 2881 samples
Epoch 1/8


array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   51,  390, 2984,  592,
       2695, 3147, 1563,  649, 1062, 1205, 2843,  674])

# LSTM

In [80]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM

In [81]:
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
token = Tokenizer(num_words=20000)
token.fit_on_texts(train_text)
x_train_seq = token.texts_to_sequences(train_text)
x_test_seq = token.texts_to_sequences(test_text)
x_train = sequence.pad_sequences(x_train_seq, maxlen = 30)
x_test = sequence.pad_sequences(x_test_seq, maxlen = 30)

In [42]:
# from sklearn.decomposition import PCA 
# pca = PCA(n_components = 20)
# x_train = pca.fit_transform(x_train)

In [82]:
model = Sequential()
model.add(Embedding(output_dim=32,
                    input_dim =20000,
                    input_length=30))
model.add(Dropout(0.3))
model.add(LSTM(32))

model.add(Dense(units=256,
                activation='relu'))
model.add(Dropout(0.20))
model.add(Dense(units=1,
                activation='sigmoid'))
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 30, 32)            640000    
_________________________________________________________________
dropout_16 (Dropout)         (None, 30, 32)            0         
_________________________________________________________________
lstm_13 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dense_13 (Dense)             (None, 256)               8448      
_________________________________________________________________
dropout_17 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 257       
Total params: 657,025
Trainable params: 657,025
Non-trainable params: 0
_________________________________________________________________


In [83]:
model.compile(loss="binary_crossentropy",
              optimizer = "adam",
              metrics=["accuracy"])
model.fit(x_train, 
          y_train, 
          batch_size=500,
          epochs=5, 
          verbose=2,
          validation_split=0.2)
scores = model.evaluate(x_test, y_test, verbose=1)
scores[1]

Train on 11523 samples, validate on 2881 samples
Epoch 1/5
 - 3s - loss: 0.6703 - acc: 0.6103 - val_loss: 0.6363 - val_acc: 0.6060
Epoch 2/5
 - 1s - loss: 0.5102 - acc: 0.7225 - val_loss: 0.3797 - val_acc: 0.8886
Epoch 3/5
 - 1s - loss: 0.2387 - acc: 0.9241 - val_loss: 0.2636 - val_acc: 0.9056
Epoch 4/5
 - 1s - loss: 0.1159 - acc: 0.9613 - val_loss: 0.2556 - val_acc: 0.9098
Epoch 5/5
 - 1s - loss: 0.0685 - acc: 0.9782 - val_loss: 0.2826 - val_acc: 0.9087
3602/3602 [==============================] - 0s 82us/step


0.90227651308140178

In [84]:
input_text=["國昌 加油 深入 民心 國家 官員 苦民所苦 以德服衆 讚 永遠 支持",
            "提到 酒駕 立院 戰神 國昌 提出 這種 意義 緊迫 法案 反觀 安安靜靜 時代力量 更多 支持",
            "謝謝 無畏 正港 政治 努力 努力 台灣 發聲 賢能",
            """黃國昌 臺灣 用心 立委 需要 時代力量 在野黨 認真 人民 發聲""",
            """立院 黃國昌 委員 委員 加油""",
            """欣賞 黃委員 認真 進度 態度""",
            """加油 委員""",
            """認真 立委 推""",
            """自己 重要 期許 完成 過去 總統 敢做 推動 過去 政府 推動 改革""",
            """讚 好棒 加油""",
           ]

In [85]:
input_seq = token.texts_to_sequences(input_text)
pad_input_seq = sequence.pad_sequences(input_seq, maxlen=30)
predict = model.predict_classes(pad_input_seq)
predict

array([[0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1]])